In [15]:
%matplotlib inline
import re
import tarfile
import os
import matplotlib.pyplot as plt
import numpy as np
import regex
import pandas as pd
from elasticsearch_dsl import *
from elasticsearch_dsl import query
from elasticsearch import *
import tqdm


In [56]:
from pathlib import Path

article_reqex = r'( *Art\. )(\d+)(\.((?! *Art)([\s\S]))*)'
pattern = re.compile(article_reqex)


article_directory = 'artykuly'

iterations = 0
path="../data/ustawy"
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r',encoding='utf-8') as file: # open in readonly mode
        content = file.read()
        file_name = Path(file.name).stem
#         print(file_name)
        for m in re.finditer(pattern, content):
#             print(m.group(1), '*', m.group(2),'*', m.group(3))
            article_filename =f'{file_name}_{m.group(2)}'
            with open(os.path.join(article_directory, article_filename), 'w',encoding='utf-8') as article_file:
                article_file.write(m.group(0))        
#         break
        iterations+=1

In [67]:
df = pd.read_csv('questions.csv',names=['passage_id', 'question', 'passage'])
print(df.passage)


0    131afsf vvsv
Name: passage, dtype: object


In [76]:
df = pd.read_csv('questions.csv',names=['passage_id', 'question', 'passage'])

for filename in os.listdir('artykuly_wybrane'):
    with open(os.path.join('artykuly_wybrane', filename), 'r',encoding='utf-8') as file: # open in readonly mode
        content = file.read()
        file_name = Path(file.name).stem
        tmp = {'passage_id': file_name, 'question': '', 'passage': content}
        df = df.append(tmp, ignore_index = True)

df.to_csv('questions.csv', index=False,header=None)

In [85]:
df = pd.read_csv('questions1.csv',names=['passage_id', 'question', 'passage'])
# df.sort_values('passage_id')
df

,passage_id,question,passage
0,2000_1319_9,Kto wydaje decyzję o uznaniu praktyki za ogran...,Art. 9.\nP...
1,2004_574_51,Kto powołuje okręgowych inspektorów rybołówstw...,Art. 51.\n...
2,2001_21_24,Czym jest park krajobrazowy?,Art. 24. 1. Park krajobrazowy jest obszarem ch...
3,1994_602_4,W jakim momencie powstaje obowiązek podatkowy?,Art. 4.\nObowi...
4,1996_465_134,W jakim momencie powstaje spółka komandytowo-a...,Art. 134.\...
5,2004_879_14,Ile przynajmniej godzin odpoczynku przysługuje...,Art. 14.\n...
6,2003_1750_112,Czy skazanie prawomocnym wyrokiem sądowym może...,Art. 112.\...
7,2001_298_39,Kogo zwalnia ze służby szef BOR?,Art. 39.\nFunk...
8,1995_368_37,Czy skarga bez wskazania określonego naruszeni...,Art. 37.\nSkar...
9,1997_944_10,Podczas jakich wydarzeń przełożony jest zobowi...,Art. 10.\n 1. ...


In [10]:

es = Elasticsearch("http://localhost:9200")
resp = es.info()
resp


{'name': 'x-dell',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'iBI8GYH1TzOb1ag0UsGzTg',
 'version': {'number': '7.10.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '1c34507e66d7db1211f66f3513706fdf548736aa',
  'build_date': '2020-12-05T01:00:33.671820Z',
  'build_snapshot': False,
  'lucene_version': '8.7.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [11]:
connections.add_connection('python_client', es)
connections.create_connection(hosts=['localhost'])

print(es.cat.plugins(params={'v': '', 'h': 'name,component,version,description'}))



name   component           version description
x-dell analysis-morfologik 7.10.1  Morfologik Polish Lemmatizer plugin for Elasticsearch



In [13]:
synonym = token_filter('my_name', type='synonym',
                       synonyms =
                       ["kpk => kodeks postępowania karnego",
                        "kpc => kodeks postępowania cywilnego",
                        "kk => kodeks karny",
                        "kc => kodeks cywilny"
                       ])

my_analyzer = analyzer('my_analyzer',
    tokenizer=tokenizer('standard'),
    filter=["lowercase",synonym,"morfologik_stem",]
)


# es.delete_by_query(index=['ustawa'], body={"query": {"match_all": {}}})


class Ustawa(Document):
    title = Text()
    created_at = Integer()
    content = Text(analyzer=my_analyzer)
    
    class Index:
        name = 'ustawa'
        
        

Ustawa.init()

iterations = 0
path="../data/ustawy"
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r',encoding='utf-8') as f: # open in readonly mode
        
        year = int(re.search(r'.*\/([^\/]{4})',f.name).group(1))
        first = Ustawa(title=f.name,content=f.read(),created_at = year)
        
        first.save(using=es)
        iterations+=1